In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

import sys

if "../.." not in sys.path:
    sys.path.insert(0, "../..")

from notebooks.utils import load_envrc

load_envrc("../../.envrc")

In [3]:
from rich import get_console
from rich.console import Console

console: Console = get_console()
console.is_jupyter = False

# PydanticAI Agents

In our previous post, we explored function calling and how it enables models to interact with external tools. It’s a powerful feature, but manually defining schemas and managing the request/response loop can become tedious as an application grows. This is where agent frameworks come in.

In this post, let’s experiment with [PydanticAI Agents](https://ai.pydantic.dev/agents/). We’ll use it to define a simple agent that mimics our earlier examples, but with  less boilerplate. We will also inspect the underlying requests to try and learn how the framework orchestrates tool calls, and finally, we'll test how well it handles swapping the backend model for a local instance running via Ollama.

## Prerequisites

Follow the README instructions to set up your Python environment.

For the code examples that use local models, we'll be using Ollama's `llama3.2` model. If you haven't done so already, [install Ollama](https://ollama.com/download) on your computer and download the model with `ollama pull llama3.2`.

### Agent: Name Cactifier

To get a feel for the syntax, let's bring back the `cactify_name` example we used in previous posts. Using PydanticAI, we can take that same logic and register it directly to the agent, bypassing the need to write out the raw schema definition manually, and letting the framework handle the request/response loop for us:

In [4]:
from pydantic_ai import Agent

agent = Agent(
    name="Name Cactifier",
    model="openai:gpt-4o-mini",
    instructions="You are a friendly agent that transforms people's names to make them more cactus-like using specific rules.",
)


@agent.tool_plain
def cactify_name(name: str) -> str:
    """Makes a name more cactus-like."""
    base_name = name
    if base_name.lower().endswith(("s", "x")):
        base_name = base_name[:-1]
    if base_name and base_name.lower()[-1] in "aeiou":
        base_name = base_name[:-1]
    return base_name + "actus"


result1 = await agent.run("What would the name Alice be if it were cactus-ified?")
print("Response:", result1.output)

Response: The cactus-ified version of the name Alice is "Alicactus"!


When we run this, the agent recognizes it needs to use the tool and returns the cactified name as expected.

That's a lot less code! But how does it work?

The [`AgentRunResult.all_messages()`](https://ai.pydantic.dev/api/agent/#pydantic_ai.agent.AgentRunResult.all_messages) method returns the conversation history as a list of [ModelMessage](https://ai.pydantic.dev/api/messages/#pydantic_ai.messages.ModelMessage) objects, which is quite verbose. To make it easier to read, we'll strip out some of the less relevant fields for brevity:

In [5]:
from notebooks.pydantic_models import print_all_messages

print_all_messages(result1.all_messages())

[
  {
    "ModelRequest": {
      "parts": [
        {
          "UserPromptPart": {
            "content": "What would the name Alice be if it were cactus-ified?"
          }
        }
      ],
      "instructions": "You are a friendly agent that transforms people's names to make them more cactus-like using specific rules."
    }
  },
  {
    "ModelResponse": {
      "parts": [
        {
          "ToolCallPart": {
            "tool_name": "cactify_name",
            "args": "{\"name\":\"Alice\"}"
          }
        }
      ]
    }
  },
  {
    "ModelRequest": {
      "parts": [
        {
          "ToolReturnPart": {
            "tool_name": "cactify_name",
            "content": "Alicactus"
          }
        }
      ],
      "instructions": "You are a friendly agent that transforms people's names to make them more cactus-like using specific rules."
    }
  },
  {
    "ModelResponse": {
      "parts": [
        {
          "TextPart": {
            "content": "The cactus-ified ver

The output shows the complete conversation flow:

1. **ModelRequest** - The user's prompt ("What would the name Alice be if it were cactus-ified?") and the models instructions.
2. **ModelResponse** - The model's decision to call the `cactify_name` tool with `{"name": "Alice"}`
3. **ModelRequest** - The tool's return value (`"Alicactus"`) sent back to the model
4. **ModelResponse** - The model's final text response incorporating the tool result

This illustrates the agent loop: user prompt → tool call → tool execution → final response.

## Llama 3.2 via Ollama

PydanticAI supports various model backends, including local models via Ollama. To switch to a local model, simply change the `model` parameter when creating the agent or when running it.

When we run the agent again with the same prompt, it successfully uses the local `llama3.2` model to cactify the name:

In [6]:
result2 = await agent.run(
    "What would the name Alice be if it were cactus-ified?",
    model="ollama:llama3.2",
)
print("Response:", result2.output)
# print_all_messages(result2.all_messages())

Response: If the original name was "Alice", the cactus-ified version is indeed "Alicactus". This transformation involves adding a "-cactus" suffix to the end of the given name, resulting in a unique and prickly cactus-inspired alternative. Would you like me to cactus-fy another name?


## OpenAI-compatible Providers

Inspecting `all_messages()` is helpful, but the abstraction can hide details. What is actually sent to the model provider?

Pydantic offers [Logfire](https://ai.pydantic.dev/logfire/), which is very useful for monitoring and debugging LLM interactions, but here we just want to log the raw requests and responses (including the payloads) to the console for a quick look. We wrote a simple helper that taps into the httpx event hooks to print the request and response data and we'll use it with our Ollama model:


In [7]:
from notebooks.pydantic_models import get_model

llama32_model_with_logging = get_model("ollama:llama3.2", debug_http=True)
result3 = await agent.run(
    "What would the name Alice be if it were cactus-ified?",
    model=llama32_model_with_logging,
)
print("Response:", result3.output)

>>> REQUEST POST http://localhost:11434/v1/chat/completions
{
  "messages": [
    {
      "content": "You are a friendly agent that transforms people's names to make them more cactus-like using specific rules.",
      "role": "system"
    },
    {
      "role": "user",
      "content": "What would the name Alice be if it were cactus-ified?"
    }
  ],
  "model": "llama3.2",
  "stream": false,
  "tool_choice": "auto",
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "cactify_name",
        "description": "Makes a name more cactus-like.",
        "parameters": {
          "additionalProperties": false,
          "properties": {
            "name": {
              "type": "string"
            }
          },
          "required": [
            "name"
          ],
          "type": "object"
        },
        "strict": true
      }
    }
  ]
}
<<< RESPONSE 200
{
  "id": "chatcmpl-702",
  "object": "chat.completion",
  "created": 1765566333,
  "model": "llama3

### Non-text response

What if our tool returned a non-text response, like a dictionary of the original name and the cactified version? Let's modify our tool to do that:

```python

In [8]:
from pydantic import BaseModel


class NameResponse(BaseModel):
    original_name: str
    cactified_name: str


agent2 = Agent(
    name="Name Cactifier",
    model="openai:gpt-4o-mini",
    instructions="You are a friendly agent that transforms people's names to make them more cactus-like using specific rules. You MUST call a tool when answering user questions if any tool is relevant. Never answer directly.",
    output_type=NameResponse,
)


@agent2.tool_plain
def cactify_name2(name: str) -> str:
    """Makes a name more cactus-like."""
    base_name = name
    if base_name.lower().endswith(("s", "x")):
        base_name = base_name[:-1]
    if base_name and base_name.lower()[-1] in "aeiou":
        base_name = base_name[:-1]
    return base_name + "actus"


result2 = await agent2.run("What would the name Alice be if it were cactus-ified?")
print("Response:", result2.output)
print_all_messages(result2.all_messages())

Response: original_name='Alice' cactified_name='Alicactus'
[
  {
    "ModelRequest": {
      "parts": [
        {
          "UserPromptPart": {
            "content": "What would the name Alice be if it were cactus-ified?"
          }
        }
      ],
      "instructions": "You are a friendly agent that transforms people's names to make them more cactus-like using specific rules. You MUST call a tool when answering user questions if any tool is relevant. Never answer directly."
    }
  },
  {
    "ModelResponse": {
      "parts": [
        {
          "ToolCallPart": {
            "tool_name": "cactify_name2",
            "args": "{\"name\":\"Alice\"}"
          }
        }
      ]
    }
  },
  {
    "ModelRequest": {
      "parts": [
        {
          "ToolReturnPart": {
            "tool_name": "cactify_name2",
            "content": "Alicactus"
          }
        }
      ],
      "instructions": "You are a friendly agent that transforms people's names to make them more cactus-l